In [9]:
import theano
import pymc3 as pm
from sklearn.cross_validation import train_test_split
# Use a theano shared variable to be able to exchange the data the model runs on
from theano import shared
import numpy as np
import pandas as pd
#构造数据集
np.random.seed(123)
alpha=1
sigma=1
beta =[1, 2.5]
N=100
X=np.random.randn(N,2)
Y=alpha + X[:,0]*beta[0]+X[:,1]*beta[1] + np.random.randn(N)*sigma
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=.2)#8成训练，2成测试
#4.2.构造贝叶斯线性回归模型
X_input = theano.shared(X_train)
Y_output = theano.shared(Y_train)
def fn_mullist(X,b):
    X_C = []
    for i in range(2):
        X_C=X[:,i]*b[i]
    return X_C
    
with pm.Model() as EDP_model:
    alpha=pm.Normal('alpha',mu=3,sd=1)#习题难度系数3作为均值
    beta=pm.Normal('beta',mu=3,sd=1,shape=2)#列，3584个beta
    sigma=pm.HalfNormal('sigma',sd=1)
    
    #mu=alpha+X_input[:,0]*beta[0]+X_input[:,1]*beta[1]  #线性回归模型
    mu=alpha+fn_mullist(X_input,beta)
    Y_obs=pm.Normal('Y_obs',mu=mu,sd=sigma,observed=Y_output)
#4.3.基于NUTS采样
with EDP_model:
    step = pm.NUTS()#step = pm.Metropolis()
    trace = pm.sample(1000,step=step)#4chains,25万/chain，共100万
pm.summary(trace)#后验统计总结
#4.4 测试集验证
X_input.set_value(X_test)
Y_output.set_value(Y_test)
with EDP_model:
    ppc = pm.sample_posterior_predictive(trace, samples=500, progressbar=False) 
pred = ppc['Y_obs'].mean(axis=0) #预测值
print('Accuracy = {}%'.format((Y_test == pred).mean() * 100))
print (Y_test-pred)

Multiprocess sampling (4 chains in 4 jobs)
NUTS: [sigma, beta, alpha]
Sampling 4 chains: 100%|██████████| 6000/6000 [00:05<00:00, 1167.31draws/s]


Accuracy = 0.0%
[-0.31298228 -0.7771232   1.60688988  1.81155645  1.33229397 -0.04769617
 -0.54223379  2.61698276 -2.49102525  0.16718957 -0.30445434  1.02902152
 -0.61907845  2.45292174 -0.07403359 -0.67787889  0.97990647 -0.12744808
 -0.84723319 -1.75561976]


In [36]:
import theano
import pymc3 as pm
from sklearn.cross_validation import train_test_split
from theano import shared
import pandas as pd
#基于贝叶斯线性回归模型预测习题难度
#1.加载训练文件
data = pd.read_csv("/data/fjs/data/math/mathV.csv",sep='|') 
#2.构造训练集和测试集
X_train, X_test, Y_train, Y_test = train_test_split(data.drop(columns=['quid','qdiff']), data[['qdiff']], test_size=.2)#8成训练，2成测试
#3.构造贝叶斯线性回归模型
X_input = theano.shared(X_train.values)#转numpy array
Y_output = theano.shared(Y_train.values)#转numpy array
#print (X_input.get_value())
def fn_mullist(X,b):
    X_C = []
    for i in range(X_train.shape[1]):
        X_C=X[:,i]*b[i]
    return X_C

with pm.Model() as EDP_model:
    alpha=pm.Normal('alpha',mu=3,sd=1)#习题难度系数3作为均值
    beta=pm.Normal('beta',mu=3,sd=1,shape=X_train.shape[1])#3列
    sigma=pm.HalfNormal('sigma',sd=1)
    
    mu=alpha+ fn_mullist(X_input.get_value(),beta) #线性回归模型
 
    Y_obs=pm.Normal('Y_obs',mu=mu,sd=sigma,observed=Y_output)
#4.基于NUTS采样
with EDP_model:
    step = pm.NUTS()#step = pm.Metropolis()
    trace = pm.sample(1000,step=step)#4chains,25万/chain，共100万
#5.测试集验证
X_input.set_value(X_test.values)
Y_output.set_value(Y_test.values)
with EDP_model:
    ppc = pm.sample_posterior_predictive(trace, samples=500, progressbar=False) 
pred = ppc['Y_obs'].mean(axis=0) #预测值
#6.评价指标
pred =pd.DataFrame(pred,columns=['Y_obs'])
def fn_typeOf(x):
    m=6
    n=0
    for i in range(6):
        if abs(x-i)<m:n=i
    return n
pred['Y_obs'] = pred['Y_obs'].apply(lambda x: fn_typeOf(x))
pred=pred['Y_obs'].tolist()
print('Accuracy = {}%'.format((Y_test == pred).mean() * 100))

FileNotFoundError: File b'/data/fjs/data/math/mathV.csv' does not exist